# Other State-of-the-Art Implementations:

## The following modules will test the other state of the art implementations in the article in the order of their appearance for 4096 pairs of nucleotide sequences:

### SSW:

In [ ]:
! time /ssw/src/ssw_test /others/data/seqx.fasta /others/data/seqy.fasta

### Parasail:

In [ ]:
! time /parasail/build/parasail_aligner -f /others/data/seqx.fasta -q /others/data/seqy.fasta

### SeqAn: Note that SeqAn was very difficult to work with external files. However the below bash command will test the package in equal settings for 48 pairs of sequences. The run time for 4096 pairs of sequences can be extrapolated by multiplying the resulting runtime by 85.3

In [ ]:
! time /seqan/build/bin/test_align_simd_global_equal_length_avx2

### Codon (seq) inter_align:

In [ ]:
! time ./seq_interalign

### ADEPT: ADEPT's test are unable to perform for arbitrary number of pairs with arbitrary lengths, and their DNA test suite of 3000 sequence pairs of length 150 was used to perform benchmarks, the resulting runtime must be multiplied by 4.66 to get the equivalent of other runtime benchmarks.

In [ ]:
! time /ADEPT/build/examples/simple_sw/simple_sw /ADEPT/test-data/dna-query.fasta /ADEPT/test-data/dna-reference.fasta out.txt res.txt

### SWIPE:

In [ ]:
! time /swipe/swipe -d /swipe/swipe_data/seqx.fasta -i /swipe/swipe_data/seqy.fasta 

### Unfortunately, we were unable to replicate 3 of the software packages we used in our benchmarks on the Docker image: RV, SW#, and Aalign.

#### 1. RV
RV needs to be interleaved with the image's LLVM build, and we could not figure out a way to make it even run in the Docker image.

#### 2. SW#
SW# utilizes CUDA 11.0 and is not compatible with CUDA 12.0 and later. However, on a Linux or Mac-based system with CUDA 11.0, SW# can be built as follows:

1. Clone the repo:
    ```bash
    git clone https://github.com/mkorpar/swsharp
    ```

2. Go to the project directory and remove the existing Makefile:
    ```bash
    cd swsharp/swsharp
    rm -rf Makefile
    ```

3. Create a new Makefile using either nano or vim and copy and paste the following script:

    ```makefile
    CC = gcc
    CP = g++
    CU = nvcc
    LD = nvcc
    DX = doxygen

    NAME = swsharp

    OBJ_DIR = obj
    SRC_DIR = src
    DOC_DIR = doc
    INC_DIR = ../include/$(NAME)
    LIB_DIR = ../lib
    EXC_DIR = ../bin
    WIN_DIR = ../swsharpwin/$(NAME)

    I_CMD = $(addprefix -I, $(SRC_DIR) )
    L_CMD = $(addprefix -L, )

    DEP_LIBS =

    CC_FLAGS = $(I_CMD) -O3 -Wall -march=native
    CP_FLAGS = $(CC_FLAGS)
    LD_FLAGS = $(I_CMD) $(L_CMD) -lpthread -lstdc++

    CU_FLAGS = $(I_CMD) -O3

    API = $(addprefix $(SRC_DIR)/, align.h alignment.h chain.h constants.h \
            cpu_module.h cuda_utils.h database.h db_alignment.h evalue.h gpu_module.h \
            post_proc.h pre_proc.h reconstruct.h scorer.h swsharp.h thread.h threadpool.h)

    SRC = $(shell find $(SRC_DIR) -type f \( -iname \*.cpp -o -iname \*.c -o -iname \*.cu \))
    HDR = $(shell find $(SRC_DIR) -type f \( -iname \*.h \))
    OBJ = $(subst $(SRC_DIR), $(OBJ_DIR), $(addsuffix .o, $(basename $(SRC))))
    DEP = $(OBJ:.o=.d)
    INC = $(subst $(SRC_DIR), $(INC_DIR), $(API))
    LIB = $(LIB_DIR)/lib$(NAME).a
    EXC = $(NAME)
    BIN = $(EXC_DIR)/$(EXC)
    DOC = $(DOC_DIR)/Doxyfile
    WIN = $(subst $(SRC_DIR), $(WIN_DIR), $(HDR) $(SRC))

    debug: CC_FLAGS := $(CC_FLAGS) -DDEBUG -DTIMERS
    debug: CP_FLAGS := $(CP_FLAGS) -DDEBUG -DTIMERS
    debug: CU_FLAGS := $(CU_FLAGS) -DDEBUG -DTIMERS --ptxas-options=-v

    cpu: LD = $(CC)

    all: $(OBJ) $(DEP_LIBS)
    debug: all
    cpu: all

    install: lib include win

    bin: $(BIN)

    include: $(INC)

    lib: $(LIB)

    win: $(WIN)

    $(EXC): $(OBJ) $(DEP_LIBS)
            @echo [LD] $@
            @mkdir -p $(dir $@)
            @$(LD) $(OBJ) -o $@ $(LD_FLAGS)
    $(OBJ_DIR)/%.o: $(SRC_DIR)/%.c
            @echo [CC] $<
            @mkdir -p $(dir $@)
            @$(CC) $< -c -o $@ -MMD $(CC_FLAGS)
    $(OBJ_DIR)/%.o: $(SRC_DIR)/%.cpp
            @echo [CP] $<
            @mkdir -p $(dir $@)
            @$(CP) $< -c -o $@ -MMD $(CP_FLAGS)

    $(OBJ_DIR)/%.o: $(SRC_DIR)/%.cu
            @mkdir -p $(dir $@)
    ifeq (,$(findstring cpu,$(MAKECMDGOALS)))
            @echo [CU] $<
            @$(CU) $< -M -o $(@:.o=.d) $(CU_FLAGS) --output-directory $(dir $@)
            @$(CU) $< -c -o $@ $(CU_FLAGS)
    else
            @echo [CP] $<
            @$(CP) -x c++ $< -c -o $@ -MMD $(CP_FLAGS)
    endif

    $(INC_DIR)/%.h: $(SRC_DIR)/%.h
            @echo [CP] $@
            @mkdir -p $(dir $@)
            @cp $< $@
            
    $(LIB): $(OBJ)
            @echo [AR] $@
            @mkdir -p $(dir $@)
            @ar rcs $(LIB) $(OBJ) 2> /dev/null

    $(BIN): $(EXC)
            @echo [CP] $@
            @mkdir -p $(dir $@)
            @cp $< $@

    $(WIN_DIR)/%: $(SRC_DIR)/%
            @echo [CP] $@
            @mkdir -p $(dir $@)
            @cp $< $@

    docs:
            @echo [DX] generating documentation
            @$(DX) $(DOC)
            
    clean:
            @echo [RM] cleaning
            @rm -rf $(OBJ_DIR) $(EXC)

    remove:
            @echo [RM] removing
            @rm -rf $(INC_DIR) $(LIB) $(BIN) $(EXC) $(WIN)

    -include $(DEP)
    ```

4. Navigate back to the main swsharp folder and run:
    ```bash
    make
    ```

5. The executables will be in `./swsharp/bin`.

6. A simple alignment test can be run by using:
    ```bash
    ./swsharp/bin/swsharpdb -i query.fasta -j target.fasta
    ```

#### 3. Aalign
Aalign utilizes `icpc` (Intel's C++ compiler), which is not available on Ubuntu 22.04's Docker image. However, on a Linux or Mac-based system with CUDA 11.0, Aalign can be built as follows:

1. Install the Intel `icpc` C++ compiler.
2. Clone the repo:
    ```bash
    git clone https://github.com/vtsynergy/aalign
    ```

3. In `./aalign/ModularDesign`, run:
    ```bash
    make
    ```

4. You will have access to Smith-Waterman (different modes) and Needleman-Wunsch (different modes).
